In [11]:
import pandas as pd
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
import json
import re
from pprint import pprint
import string
import sys
pd.options.display.max_colwidth=150
pd.set_option('display.max_rows', 3000)
products = []
listings = []
with open('data/products.txt') as products_file:
    for line in products_file:
        products.append(json.loads(line))
with open('data/listings.txt') as listings_file:
    for line in listings_file:
        listings.append(json.loads(line))
#pprint(products)
df_products = pd.DataFrame(products, columns=['product_name', 'manufacturer', 'family', 'model'])
df_listings = pd.DataFrame(listings, columns=['title', 'manufacturer', 'price', 'currency'])
df_products = df_products.fillna('')

In [15]:
original_list_idx_1stmatch = \
[8396, 11918, 7634, 8333, 19079, 8332, 11864, 19434, 7595, 7887, 7425, 6483, 6482, 11886, 16456, 15707, 12263, 12609, 12236, 8411, 759, 7596, 10938, 19074, 7633, 18340, 15706, 12306, 12305, 10900, 11395, 10166, 10442, 12347, 2200, 2199, 2196, 10441, 10986, 2112, 18461, 18460, 19193, 10866, 18389, 18390, 2182, 11541, 10867, 11047, 728, 10163, 749, 748, 751, 18378, 2107, 10162, 19187, 9773, 12005, 729, 10161, 19994, 747, 2208, 2202, 2201, 9702, 18384, 10407, 18372, 10868, 2174, 19769, 19747, 20151, 10985, 11989, 2124, 19770, 19766, 19756, 2159, 19758, 2153, 10200, 2128, 2129, 10983, 10893, 19429, 2137, 2143, 19763, 18398, 1217, 2176, 2146, 2175, 10899, 10186, 19764, 20150, 10187, 19428, 11206, 2180, 2172, 2117, 2120, 18453, 11287, 20159, 2151, 19750, 10167, 11401, 11021, 19753, 19917, 2121, 10199, 1228, 2160, 2123, 11996, 19198, 11474, 19768, 19921, 19922, 11980, 11995, 735, 10168, 18452, 1428, 1201, 10944, 11419, 10945, 9596, 10228, 1124, 18189, 11245, 18188, 9595, 9594, 10937, 1045, 2556, 19355, 255, 11249, 2564, 9584, 9583, 2576, 2578, 9582, 11881, 10238, 10308, 18202, 19838, 19827, 1145, 1144, 2475, 10338, 20001, 10926, 19829, 19369, 1141, 1140, 19839, 11486, 18216, 18215, 10317, 10931, 2506, 2509, 10932, 1125, 11031, 2517, 10936, 10227, 2465, 2581, 12462, 10957, 9559, 11894, 9558, 12470, 2658, 10288, 12476, 2677, 18109, 19872, 10956, 10287, 2692, 2700, 11040, 232, 18092, 19880, 18098, 18097, 2707, 11904, 19881, 12480, 2585, 247, 19320, 10307, 2587, 11252, 2601, 2613, 10239, 10300, 2616, 19316, 19851, 10296, 10295, 2620, 12463, 2623, 19860, 2639, 19864, 18137, 20009, 2642, 9560, 19867, 19319, 19858, 19904, 9633, 19375, 2247, 2251, 710, 9699, 709, 708, 10405, 11030, 19402, 19799, 11528, 10901, 19804, 11957, 19249, 20000, 2272, 19250, 11521, 10216, 2277, 19391, 2278, 2282, 231, 19390, 19998, 9700, 1200, 1199, 11407, 11046, 1198, 11281, 1196, 1195, 716, 11409, 11832, 11969, 2231, 9701, 715, 1194, 11223, 11029, 19785, 712, 711, 10406, 11041, 2234, 11027, 2286, 10914, 2287, 1173, 1172, 688, 2390, 687, 2393, 686, 2394, 2412, 11949, 18254, 10925, 1149, 1148, 1147, 18247, 19380, 10343, 685, 10982, 10342, 9635, 1146, 9634, 2426, 18267, 11243, 11241, 19258, 1185, 2303, 2304, 10915, 2309, 1184, 2310, 10217, 1180, 1179, 18294, 2334, 2344, 2345, 2352, 35, 10218, 2371, 2372, 2373, 34, 19385, 19817, 1174, 2220, 1186, 18561, 12007, 12207, 19580, 1663, 1341, 12208, 1664, 1665, 1340, 18827, 1666, 1667, 1671, 1674, 1656, 1682, 1687, 1339, 11130, 11132, 20103, 20104, 1690, 1691, 973, 974, 18815, 1693, 19588, 1684, 19589, 12206, 1649, 10090, 12179, 20097, 18879, 12071, 18878, 12183, 19570, 1360, 12184, 12187, 19571, 1359, 1651, 20098, 1353, 19574, 1619, 1621, 1625, 1352, 1348, 1641, 1642, 1646, 1647, 18840, 19579, 1358, 9955, 11336, 18813, 1327, 10710, 19080, 10711, 11007, 979, 1742, 11324, 19964, 1326, 19962, 11006, 11060, 978, 12054, 11005, 11160, 12242, 18758, 1762, 12244, 11452, 19615, 12245, 11453, 19617, 1318, 9854, 1747, 11335, 977, 1736, 18812, 12211, 1704, 1705, 19590, 12214, 975, 19591, 19592, 19593, 11446, 19969, 10680, 1328, 10681, 19602, 18792, 18790, 11447, 1721, 12063, 1723, 18784, 18783, 10694, 11073, 20113, 976, 11328, 10999, 20095, 1582, 20077, 1479, 10034, 11742, 11768, 1414, 12124, 12125, 10537, 1484, 10033, 18963, 11356, 1477, 10032, 1412, 1411, 1408, 19014, 1494, 1495, 1499, 18953, 19015, 18952, 12132, 19017, 12092, 10538, 1403, 20076, 1415, 11361, 1431, 11360, 20072, 12105, 10522, 20073, 12106, 12107, 1439, 19981, 1451, 1452, 10531, 19980, 11440, 138, 18987, 1460, 12101, 20074, 20075, 11096, 10528, 10529, 11751, 10530, 11749, 10050, 1366, 18948, 10584, 20087, 1535, 20088, 1381, 896, 10606, 12084, 12161, 18910, 972, 892, 19043, 18907, 1382, 18906, 10637, 11079, 1566, 1567, 12171, 20093, 19045, 1575, 20094, 9961, 10089, 9960, 1580, 10636, 10583, 897, 899, 18944, 11352, 11082, 10019, 10585, 11769, 11770, 1509, 12139, 10018, 20084, 1399, 1395, 898, 10597, 1519, 1520, 1394, 902, 1393, 12091, 901, 900, 10598, 18928, 1384, 10599, 19033, 903, 1765, 1768, 11320, 10131, 1261, 10843, 2003, 1260, 10844, 18574, 2005, 19716, 764, 1259, 2007, 2011, 19712, 10132, 2016, 10049, 2019, 2023, 10133, 10134, 12301, 18556, 18555, 9799, 2030, 2037, 19159, 2015, 2040, 1999, 19147, 11184, 12289, 18607, 1275, 19939, 18605, 1969, 1972, 1973, 1974, 10126, 1975, 12025, 1996, 1977, 19938, 10837, 18591, 1983, 18588, 18587, 1988, 1989, 19708, 10127, 10130, 20146, 19711, 11434, 1964, 9798, 9788, 2074, 19177, 11055, 760, 9777, 2075, 758, 2076, 2077, 2079, 2080, 2082, 2083, 2073, 2084, 2086, 2087, 2088, 2089, 19729, 2090, 2091, 2093, 2095, 2096, 2097, 2099, 2101, 2085, 10852, 2072, 19176, 19167, 18544, 2042, 2044, 19723, 2045, 2047, 2048, 2049, 2052, 2053, 2055, 9787, 2071, 2057, 2059, 19932, 2061, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 12311, 2070, 19175, 9786, 18613, 1958, 1955, 19640, 10503, 10727, 11632, 18708, 1833, 20123, 20124, 1308, 1836, 1838, 1842, 20125, 10726, 19102, 1843, 1859, 1860, 1864, 19646, 1871, 18685, 1872, 10755, 805, 1873, 10493, 10482, 18696, 1880, 1311, 19635, 1769, 11650, 11649, 11319, 9853, 1317, 11059, 11058, 1316, 19627, 18737, 12053, 18735, 1818, 19628, 185, 1793, 18727, 19631, 1795, 1797, 18721, 18720, 19632, 19633, 1805, 19634, 1312, 10998, 1882, 18676, 11056, 10474, 785, 784, 20135, 20136, 11312, 783, 1279, 192, 18629, 782, 19947, 1944, 10785, 19455, 19454, 1277, 10802, 10803, 10804, 10805, 19133, 11311, 11176, 10811, 1276, 19943, 19696, 1278, 10475, 20134, 1940, 10762, 10765, 1302, 1300, 1299, 10769, 1893, 11393, 1894, 10481, 1897, 19119, 19465, 19464, 18662, 1284, 797, 10480, 10478, 1918, 1923, 1925, 11314, 11612, 11611, 1937, 1939, 19458, 10784, 11020, 11000, 3754, 2756, 13642, 5021, 5020, 5019, 5018, 13641, 8305, 5012, 16422, 4989, 4985, 4984, 4981, 4979, 4978, 8313, 4977, 8325, 16451, 4969, 4968, 16404, 4967, 13648, 16394, 5113, 8266, 5093, 5092, 5091, 5068, 5067, 5065, 5064, 8267, 5063, 8268, 8269, 8275, 8276, 5038, 5037, 5036, 5035, 5034, 5033, 8287, 5114, 8326, 4957, 16546, 16547, 4855, 16554, 4850, 8406, 4843, 8407, 8414, 8415, 4833, 4832, 4831, 4830, 4827, 8429, 16579, 4799, 4798, 4795, 8438, 16543, 8342, 16542, 4862, 8349, 8359, 8360, 16473, 4945, 4939, 4938, 4935, 4934, 16491, 8385, 4912, 8386, 8387, 4893, 4892, 16525, 8390, 4875, 4874, 4863, 4857, 5118, 5125, 5126, 5457, 5456, 5450, 5449, 5441, 5440, 16124, 5439, 5435, 5432, 5431, 5430, 8125, 16142, 16143, 8143, 5400, 5399, 5391, 8144, 5383, 5462, 13762, 5463, 8113, 13838, 16049, 5555, 13837, 13836, 8075, 16061, 16062, 8076, 16078, 16079, 5507, 5504, 13803, 5499, 13802, 13801, 5487, 16102, 8111, 8112, 5472, 13761, 8147, 8148, 5193, 5192, 16269, 5191, 8180, 8181, 16274, 16276, 16280, 5182, 13716, 5171, 5160, 5159, 8230, 5151, 8231, 5142, 5141, 5140, 5127, 5195, 16263, 5200, 16241, 5362, 5343, 5337, 5336, 5335, 5330, 5307, 5296, 5295, 16197, 8439, 5280, 5274, 5262, 13751, 5261, 5260, 13750, 5246, 5244, 5240, 5234, 5275, 5556, 8443, 13496, 16858, 16859, 8668, 8669, 4541, 4540, 4536, 13327, 4514, 8710, 13314, 4503, 16882, 16883, 4502, 13311, 8715, 8716, 8718, 8727, 4477, 13339, 16900, 8648, 13362, 16802, 16803, 16804, 16805, 8608, 13388, 4578, 4577, 13386, 8611, 8612, 13382, 4570, 4569, 16824, 16825, 4561, 13371, 13366, 13365, 13363, 13359, 13397, 4459, 8735, 8774, 4373, 8776, 8779, 4359, 4353, 8793, 8794, 4339, 16980, 13249, 8797, 4318, 16987, 16988, 8803, 13245, 4307, 4306, 4302, 4301, 13273, 4458, 4393, 4396, 4457, 4456, 16911, 16912, 13292, 8747, 4447, 4446, 13286, 8751, 16924, 16925, 16926, 8756, 4423, 4414, 16939, 16940, 4407, 4406, 4405, 4395, 16799, 16798, 16797, 16646, 4733, 16648, 4732, 16650, 13484, 4730, 4719, 4718, 4717, 4714, 8480, 4701, 4700, 4699, 4698, 4692, 4691, 8549, 13451, 13450, 2745, 4686, 4747, 16641, 16600, 16605, 4777, 16617, 13489, 4774, 4773, 4772, 4768, 4767, 13488, 13487, 13486, 16627, 4758, 4754, 4753, 4752, 13485, 4750, 4749, 4748, 4680, 16692, 16693, 4608, 16764, 16765, 16766, 8597, 8598, 13412, 13411, 16778, 16779, 16780, 13406, 16784, 16785, 4594, 13402, 13400, 13399, 16794, 16795, 16796, 8595, 8594, 4609, 13421, 4676, 4675, 8550, 8551, 16704, 8552, 16709, 8553, 8554, 4655, 13497, 4654, 4650, 4649, 8575, 13434, 8576, 13432, 13431, 8586, 8587, 4620, 4653, 13240, 16045, 8055, 6973, 6962, 14365, 14364, 14363, 6947, 6944, 7569, 15041, 6928, 7589, 6921, 6920, 15059, 15068, 6907, 15074, 15075, 7612, 14306, 14305, 14375, 7616, 14376, 15009, 14932, 14933, 14934, 7059, 7058, 14416, 14415, 14412, 14411, 14410, 14960, 7537, 14969, 14970, 14971, 14398, 14397, 14379, 14378, 14377, 15008, 15010, 14931, 6822, 7625, 6711, 6709, 14262, 6699, 15226, 15227, 7649, 6686, 14254, 14253, 7653, 6676, 6672, 6669, 15260, 15272, 15279, 15284, 6621, 7684, 15293, 14263, 7624, 14266, 15204, 6816, 6815, 7626, 15133, 15134, 7632, 15149, 14285, 14284, 14283, 14279, 14278, 14277, 7642, 15177, 15179, 6750, 15188, 15189, 14268, 14267, 15205, 14930, 7514, 7511, 14697, 14698, 14704, 14705, 14706, 7237, 14711, 14712, 14713, 14714, 7229, 14721, 14722, 14726, 14727, 14729, 7217, 14737, 14738, 14739, 14740, 14673, 14741, 14672, 7409, 7325, 14565, 14587, 14588, 14564, 14562, 14561, 14560, 14601, 14602, 14603, 7316, 14545, 14544, 7376, 14530, 7269, 14529, 14528, 14527, 7408, 14671, 7209, 7191, 14774, 7105, 14461, 14460, 7100, 7099, 14459, 7090, 14447, 7087, 14446, 14445, 7510, 14437, 14436, 14435, 14434, 14908, 14909, 14433, 7076, 14432, 14869, 14868, 7125, 14852, 14775, 7453, 14796, 14797, 14798, 14799, 14803, 14804, 14805, 14823, 15294, 14824, 14828, 14487, 14833, 14834, 14835, 14836, 7138, 14849, 14850, 14851, 14827, 5559, 14207, 14206, 13967, 13966, 7931, 15803, 5917, 7952, 5913, 5912, 5906, 5905, 5902, 13952, 13951, 13950, 13949, 13948, 5885, 5882, 13947, 5876, 13946, 13968, 13945, 13969, 7927, 15725, 6022, 6020, 6011, 15733, 15734, 15736, 15737, 15742, 5980, 15747, 5979, 5978, 5971, 5970, 5969, 5965, 5964, 5963, 7922, 7923, 15794, 6027, 13944, 5852, 5640, 5639, 5638, 5633, 15982, 15983, 5614, 15991, 15992, 13865, 5602, 13864, 13863, 13862, 5593, 13861, 16011, 16020, 5577, 8053, 8054, 5645, 13943, 5648, 8034, 5813, 5801, 5800, 5793, 5788, 5776, 7994, 13901, 13900, 13899, 5736, 5735, 5719, 5718, 5715, 5710, 8018, 8019, 15952, 15953, 5666, 8035, 6030, 6031, 15720, 15412, 6490, 14147, 6472, 6471, 14146, 6450, 6449, 7760, 7762, 14139, 14138, 7763, 6414, 7765, 7766, 7771, 6395, 6394, 14126, 14125, 6495, 6387, 6496, 6500, 15313, 15314, 7703, 7704, 6593, 15322, 15324, 6591, 6570, 15359, 6559, 14171, 14170, 6553, 14163, 14162, 14161, 15384, 6525, 7743, 6501, 6497, 14124, 6377, 6376, 6114, 15648, 15649, 7878, 6077, 14013, 14012, 14011, 14010, 14006, 14005, 14004, 14003, 13998, 13997, 6045, 6044, 6043, 6042, 6035, 6034, 6116, 6117, 7848, 6123, 14115, 14114, 14113, 15519, 15520, 6306, 14089, 15557, 15558, 15559, 15309, 15560, 7803, 6234, 14062, 14061, 14060, 6143, 14047, 15625, 15626, 14046, 14088, 4295, 4742, 4291, 12899, 9127, 3580, 3579, 9129, 9132, 9133, 9134, 17560, 4294, 3532, 3531, 3530, 3529, 3528, 3527, 9145, 12860, 12865, 12855, 17559, 9123, 3635, 3632, 12919, 3629, 3628, 3627, 3626, 3625, 17558, 3624, 9109, 12917, 12913, 3594, 3593, 3592, 3591, 9122, 3623, 9149, 9154, 12843, 9182, 3453, 17688, 17689, 3452, 17699, 17700, 17701, 9181, 3402, 3395, 3394, 17710, 17711, 12756, 3381, 3380, 3374, 3401, 9180, 3474, 17673, 9155, 9163, 3507, 3506, 3505, 9171, 17650, 17651, 17652, 17653, 9172, 17655, 17656, 17660, 17661, 9177, 3476, 3475, 17672, 3636, 3637, 3638, 12920, 17396, 17397, 3739, 17408, 17409, 3734, 3733, 3732, 17395, 3727, 12981, 9046, 9049, 9051, 9054, 3705, 9055, 12967, 3726, 3740, 13003, 17388, 17344, 9001, 3817, 3816, 17352, 17353, 17354, 3815, 3814, 17368, 9009, 9010, 9011, 9012, 13012, 9015, 9016, 3753, 17387, 3704, 3373, 9058, 17443, 9077, 3658, 3657, 9081, 12934, 12933, 3645, 3644, 12941, 9095, 17512, 17513, 3643, 9098, 9102, 9105, 12922, 9108, 12927, 9076, 12943, 17476, 17444, 9059, 17448, 17449, 3690, 3689, 12953, 12952, 3688, 3687, 3679, 3678, 12950, 12948, 9070, 17469, 17470, 9075, 17475, 3703, 17343, 3372, 17727, 17963, 12579, 3068, 3067, 9430, 9435, 9436, 9437, 9407, 17979, 3013, 3012, 3011, 3010, 12558, 12553, 12552, 12535, 17980, 2975, 9406, 12593, 3148, 3147, 17922, 17923, 9388, 12615, 12614, 3130, 9405, 3125, 17937, 12600, 17942, 3101, 17950, 12594, 3097, 3096, 3124, 2974, 12525, 18018, 18065, 18066, 18067, 18068, 2811, 2810, 2809, 2803, 18064, 2785, 2784, 18076, 18077, 18078, 18079, 2781, 2779, 2764, 12492, 9531, 2827, 2829, 12508, 2935, 2934, 2933, 18028, 2914, 2913, 2882, 2881, 2879, 12500, 12499, 2873, 2868, 9530, 2845, 2844, 2832, 18057, 3149, 9387, 3165, 3166, 3301, 17787, 17788, 12697, 12693, 17796, 17799, 17800, 3302, 12683, 3271, 17816, 17818, 17819, 17820, 17823, 3263, 3262, 12679, 3303, 17781, 17780, 17728, 3365, 3364, 3363, 12754, 12753, 9197, 3362, 3361, 17740, 17741, 12725, 3340, 3339, 3317, 12716, 12715, 12713, 12712, 17828, 12755, 17829, 3243, 17875, 3188, 3187, 3185, 17883, 17884, 3184, 3178, 3189, 3177, 3175, 3174, 3173, 3172, 3169, 12625, 3168, 3167, 3176, 3190, 3191, 3196, 17835, 17836, 3242, 17839, 3241, 3240, 3235, 3234, 3232, 3231, 3228, 3227, 3226, 17857, 17861, 3207, 3203, 3202, 3197, 12650, 3818, 6021, 17172, 13099, 3914, 4232, 4203, 4204, 8944, 3915, 17073, 17257, 13110, 3919, 17258, 17184, 17185, 3921, 13113, 13212, 17096, 17193, 8963, 3920, 17181, 4233, 3958, 8977, 4197, 3887, 3888, 4134, 4262, 4251, 17095, 8877, 3973, 13059, 8926, 13060, 4050, 13061, 4250, 8923, 17180, 8964, 13202, 4202, 13066, 17194, 17227, 13211, 13204, 17101, 13126, 13125, 8956, 17058, 4211, 3953, 8954, 13119, 17272, 17217, 17218, 17061, 4135, 17059, 8946, 8912, 17100, 17060, 13095, 17282, 17169, 13205, 17283, 8828, 13208, 4205, 17114, 17199, 17200, 17226, 17203, 13072, 3989, 17111, 4225, 4224, 8957, 17110, 17204, 17102, 3990, 13129, 4212, 17207, 17208, 8937, 4198, 3840, 17156, 3839, 17157, 4073, 4072, 17168, 13201, 8986, 13177, 3857, 3858, 3859, 8983, 4138, 4137, 13047, 13228, 17010, 17161, 17071, 3838, 17146, 17072, 17145, 4111, 8987, 4101, 13237, 17074, 3827, 17248, 13236, 4098, 4112, 4097, 17009, 4184, 3828, 4183, 3878, 13115, 13226, 3959, 8822, 4266, 17247, 17126, 13224, 17125, 13225, 17115, 13179, 8868, 17093, 8940, 4136, 17128, 3879, 17162, 3962, 4265, 3880, 4071, 17127, 3881, 12150, 9548, 9684, 9844, 9742, 9685, 9743, 10247, 9845, 9632, 10314, 9631, 10313, 9683, 9846, 9433, 9946, 9393, 9562, 9770, 9434, 9769, 9819, 9847, 10368, 9382, 9383, 11463, 9394, 9518, 10016, 10015, 9945, 10656, 9687, 9519, 11464, 9361, 9549, 9970, 9663, 10655, 9686, 9662, 9313, 9969, 9360, 10369, 9820, 9314, 10223, 9930, 9561, 1476, 1792, 1596, 2714, 2588, 1824, 1528, 1891, 2017, 1377, 1402, 1892, 1851, 2718, 12724, 12072, 12563, 12473, 1164, 1471, 2519, 7532]
#print(original_list_idx_1stmatch)
#print(len(original_list_idx_1stmatch))

In [16]:
(df_listings.iloc[original_list_idx_1stmatch,:]);

In [13]:
list_idx_1stmatch = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 17245, 17246, 17247, 17248, 17249, 17250, 17251, 17252, 17253, 17254, 17255, 17256, 17257, 17258, 17259, 17260, 17261, 17262, 17263, 17264, 17265, 17266, 17267, 17268, 17269, 17270, 17271, 17272, 17273, 17274, 17275, 17276, 17277, 17278, 17279, 17280, 17281, 17282, 17283, 17284, 17285, 17286, 17287, 17288, 17289, 17290, 17291, 17292, 17293, 17294, 17295, 17296, 17297, 17298, 17299, 17300, 17301, 17302, 17303, 17304, 17305, 17306, 17307, 17308, 17309, 17310, 17311, 17312, 17313, 17314, 17315, 17316, 17317, 17318, 17319, 17320, 17321, 17322, 17323, 17324, 17325, 17326, 17327, 17328, 17329, 17330, 17331, 17332, 17333, 17334, 17335, 17336, 17337, 17338, 17339, 17340, 17341, 17342, 17343, 17344, 17345, 17346, 17347, 17348, 17349, 17350, 17351, 17352, 17353, 17354, 17355, 17356, 17357, 17358, 17359, 17360, 17361, 17362, 17363, 17364, 17365, 17366, 17367, 17368, 17369, 17370, 17371, 17372, 17373, 17374, 17375, 17376, 17377, 17378, 17379, 17380, 17381, 17382, 17383, 17384, 17385, 17386, 17387, 17388, 17389, 17390, 17391, 17392, 17393, 17394, 17395, 17396, 17397, 17398, 17399, 17400, 17401, 17402, 17403, 17404, 17405, 17406, 17407, 17408, 17409, 17410, 17411, 17412, 17413, 17414, 17415, 17416, 17417, 17418, 17419, 17420, 17421, 17422, 17423, 17424, 17425, 17426, 17427, 17428, 17429, 17430, 17431, 17432, 17433, 17434, 17435, 17436, 17437, 17438, 17439, 17440, 17441, 17442, 17443, 17444, 17445, 17446, 17447, 17448, 17449, 17450, 17451, 17452, 17453, 17454, 17455, 17456, 17457, 17458, 17459, 17460, 17461, 17462, 17463, 17464, 17465, 17466, 17467, 17468, 17469, 17470, 17471, 17472, 17473, 17474, 17475, 17476, 17477, 17478, 17479, 17480, 17481, 17482, 17483, 17484, 17485, 17486, 17487, 17488, 17489, 17490, 17491, 17492, 17493, 17494, 17495, 17496, 17497, 17498, 17499, 17500, 17501, 17502, 17503, 17504, 17505, 17506, 17507, 17508, 17509, 17510, 17511, 17512, 17513, 17514, 17515, 17516, 17517, 17518, 17519, 17520, 17521, 17522, 17523, 17524, 17525, 17526, 17527, 17528, 17529, 17530, 17531, 17532, 17533, 17534, 17535, 17536, 17537, 17538, 17539, 17540, 17541, 17542, 17543, 17544, 17545, 17546, 17547, 17548, 17549, 17550, 17551, 17552, 17553, 17554, 17555, 17556, 17557, 17558, 17559, 17560, 17561, 17562, 17563, 17564, 17565, 17566, 17567, 17568, 17569, 17570, 17571, 17572, 17573, 17574, 17575, 17576, 17577, 17578, 17579, 17580, 17581, 17582, 17583, 17584, 17585, 17586, 17587, 17588, 17589, 17590, 17591, 17592, 17593, 17594, 17595, 17596, 17597, 17598, 17599, 17600, 17601, 17602, 17603, 17604, 17605, 17606, 17607, 17608, 17609, 17610, 17611, 17612, 17613, 17614, 17615, 17616, 17617, 17618, 17619, 17620, 17621, 17622, 17623, 17624, 17625, 17626, 17627, 17628, 17629, 17630, 17631, 17632, 17633, 17634, 17635, 17636, 17637, 17638, 17639, 17640, 17641, 17642, 17643, 17644, 17645, 17646, 17647, 17648, 17649, 17650, 17651, 17652, 17653, 17654, 17655, 17656, 17657, 17658, 17659, 17660, 17661, 17662, 17663, 17664, 17665, 17666, 17667, 17668, 17669, 17670, 17671, 17672, 17673, 17674, 17675, 17676, 17677, 17678, 17679, 17680, 17681, 17682, 17683, 17684, 17685, 17686, 17687, 17688, 17689, 17690, 17691, 17692, 17693, 17694, 17695, 17696, 17697, 17698, 17699, 17700, 17701, 17702, 17703, 17704, 17705, 17706, 17707, 17708, 17709, 17710, 17711, 17712, 17713, 17714, 17715, 17716, 17717, 17718, 17719, 17720, 17721, 17722, 17723, 17724, 17725, 17726, 17727, 17728, 17729, 17730, 17731, 17732, 17733, 17734, 17735, 17736, 17737, 17738, 17739, 17740, 17741, 17742, 17743, 17744, 17745, 17746, 17747, 17748, 17749, 17750, 17751, 17752, 17753, 17754, 17755, 17756, 17757, 17758, 17759, 17760, 17761, 17762, 17763, 17764, 17765, 17766, 17767, 17768, 17769, 17770, 17771, 17772, 17773, 17774, 17775, 17776, 17777, 17778, 17779, 17780, 17781, 17782, 17783, 17784, 17785, 17786, 17787, 17788, 17789, 17790, 17791, 17792, 17793, 17794, 17795, 17796, 17797, 17798, 17799, 17800, 17801, 17802, 17803, 17804, 17805, 17806, 17807, 17808, 17809, 17810, 17811, 17812, 17813, 17814, 17815, 17816, 17817, 17818, 17819, 17820, 17821, 17822, 17823, 17824, 17825, 17826, 17827, 17828, 17829, 17830, 17831, 17832, 17833, 17834, 17835, 17836, 17837, 17838, 17839, 17840, 17841, 17842, 17843, 17844, 17845, 17846, 17847, 17848, 17849, 17850, 17851, 17852, 17853, 17854, 17855, 17856, 17857, 17858, 17859, 17860, 17861, 17862, 17863, 17864, 17865, 17866, 17867, 17868, 17869, 17870, 17871, 17872, 17873, 17874, 17875, 17876, 17877, 17878, 17879, 17880, 17881, 17882, 17883, 17884, 17885, 17886, 17887, 17888, 17889, 17890, 17891, 17892, 17893, 17894, 17895, 17896, 17897, 17898, 17899, 17900, 17901, 17902, 17903, 17904, 17905, 17906, 17907, 17908, 17909, 17910, 17911, 17912, 17913, 17914, 17915, 17916, 17917, 17918, 17919, 17920, 17921, 17922, 17923, 17924, 17925, 17926, 17927, 17928, 17929, 17930, 17931, 17932, 17933, 17934, 17935, 17936, 17937, 17938, 17939, 17940, 17941, 17942, 17943, 17944, 17945, 17946, 17947, 17948, 17949, 17950, 17951, 17952, 17953, 17954, 17955, 17956, 17957, 17958, 17959, 17960, 17961, 17962, 17963, 17964, 17965, 17966, 17967, 17968, 17969, 17970, 17971, 17972, 17973, 17974, 17975, 17976, 17977, 17978, 17979, 17980, 17981, 17982, 17983, 17984, 17985, 17986, 17987, 17988, 17989, 17990, 17991, 17992, 17993, 17994, 17995, 17996, 17997, 17998, 17999, 18000, 18001, 18002, 18003, 18004, 18005, 18006, 18007, 18008, 18009, 18010, 18011, 18012, 18013, 18014, 18015, 18016, 18017, 18018, 18019, 18020, 18021, 18022, 18023, 18024, 18025, 18026, 18027, 18028, 18029, 18030, 18031, 18032, 18033, 18034, 18035, 18036, 18037, 18038, 18039, 18040, 18041, 18042, 18043, 18044, 18045, 18046, 18047, 18048, 18049, 18050, 18051, 18052, 18053, 18054, 18055, 18056, 18057, 18058, 18059, 18060, 18061, 18062, 18063, 18064, 18065, 18066, 18067, 18068, 18069, 18070, 18071, 18072, 18073, 18074, 18075, 18076, 18077, 18078, 18079, 18080, 18081, 18082, 18083, 18084, 18085, 18086, 18087, 18088, 18089, 18090, 18091, 18092, 18093, 18094, 18095, 18096, 18097, 18098, 18099, 18100, 18101, 18102, 18103, 18104, 18105, 18106, 18107, 18108, 18109, 18110, 18111, 18112, 18113, 18114, 18115, 18116, 18117, 18118, 18119, 18120, 18121, 18122, 18123, 18124, 18125, 18126, 18127, 18128, 18129, 18130, 18131, 18132, 18133, 18134, 18135, 18136, 18137, 18138, 18139, 18140, 18141, 18142, 18143, 18144, 18145, 18146, 18147, 18148, 18149, 18150, 18151, 18152, 18153, 18154, 18155, 18156, 18157, 18158, 18159, 18160, 18161, 18162, 18163, 18164, 18165, 18166, 18167, 18168, 18169, 18170, 18171, 18172, 18173, 18174, 18175, 18176, 18177, 18178, 18179, 18180, 18181, 18182, 18183, 18184, 18185, 18186, 18187, 18188, 18189, 18190, 18191, 18192, 18193, 18194, 18195, 18196, 18197, 18198, 18199, 18200, 18201, 18202, 18203, 18204, 18205, 18206, 18207, 18208, 18209, 18210, 18211, 18212, 18213, 18214, 18215, 18216, 18217, 18218, 18219, 18220, 18221, 18222, 18223, 18224, 18225, 18226, 18227, 18228, 18229, 18230, 18231, 18232, 18233, 18234, 18235, 18236, 18237, 18238, 18239, 18240, 18241, 18242, 18243, 18244, 18245, 18246, 18247, 18248, 18249, 18250, 18251, 18252, 18253, 18254, 18255, 18256, 18257, 18258, 18259, 18260, 18261, 18262, 18263, 18264, 18265, 18266, 18267, 18268, 18269, 18270, 18271, 18272, 18273, 18274, 18275, 18276, 18277, 18278, 18279, 18280, 18281, 18282, 18283, 18284, 18285, 18286, 18287, 18288, 18289, 18290, 18291, 18292, 18293, 18294, 18295, 18296, 18297, 18298, 18299, 18300, 18301, 18302, 18303, 18304, 18305, 18306, 18307, 18308, 18309, 18310, 18311, 18312, 18313, 18314, 18315, 18316, 18317, 18318, 18319, 18320, 18321, 18322, 18323, 18324, 18325, 18326, 18327, 18328, 18329, 18330, 18331, 18332, 18333, 18334, 18335, 18336, 18337, 18338, 18339, 18340, 18341, 18342, 18343, 18344, 18345, 18346, 18347, 18348, 18349, 18350, 18351, 18352, 18353, 18354, 18355, 18356, 18357, 18358, 18359, 18360, 18361, 18362, 18363, 18364, 18365, 18366, 18367, 18368, 18369, 18370, 18371, 18372, 18373, 18374, 18375, 18376, 18377, 18378, 18379, 18380, 18381, 18382, 18383, 18384, 18385, 18386, 18387, 18388, 18389, 18390, 18391, 18392, 18393, 18394, 18395, 18396, 18397, 18398, 18399, 18400, 18401, 18402, 18403, 18404, 18405, 18406, 18407, 18408, 18409, 18410, 18411, 18412, 18413, 18414, 18415, 18416, 18417, 18418, 18419, 18420, 18421, 18422, 18423, 18424, 18425, 18426, 18427, 18428, 18429, 18430, 18431, 18432, 18433, 18434, 18435, 18436, 18437, 18438, 18439, 18440, 18441, 18442, 18443, 18444, 18445, 18446, 18447, 18448, 18449, 18450, 18451, 18452, 18453, 18454, 18455, 18456, 18457, 18458, 18459, 18460, 18461, 18462, 18463, 18464, 18465, 18466, 18467, 18468, 18469, 18470, 18471, 18472, 18473, 18474, 18475, 18476, 18477, 18478, 18479, 18480, 18481, 18482, 18483, 18484, 18485, 18486, 18487, 18488, 18489, 18490, 18491, 18492, 18493, 18494, 18495, 18496, 18497, 18498, 18499, 18500, 18501, 18502, 18503, 18504, 18505, 18506, 18507, 18508, 18509, 18510, 18511, 18512, 18513, 18514, 18515, 18516, 18517, 18518, 18519, 18520, 18521, 18522, 18523, 18524, 18525, 18526, 18527, 18528, 18529, 18530, 18531, 18532, 18533, 18534, 18535, 18536, 18537, 18538, 18539, 18540, 18541, 18542, 18543, 18544, 18545, 18546, 18547, 18548, 18549, 18550, 18551, 18552, 18553, 18554, 18555, 18556, 18557, 18558, 18559, 18560, 18561, 18562, 18563, 18564, 18565, 18566, 18567, 18568, 18569, 18570, 18571, 18572, 18573, 18574, 18575, 18576, 18577, 18578, 18579, 18580, 18581, 18582, 18583, 18584, 18585, 18586, 18587, 18588, 18589, 18590, 18591, 18592, 18593, 18594, 18595, 18596, 18597, 18598, 18599, 18600, 18601, 18602, 18603, 18604, 18605, 18606, 18607, 18608, 18609, 18610, 18611, 18612, 18613, 18614, 18615, 18616, 18617, 18618, 18619, 18620, 18621, 18622, 18623, 18624, 18625, 18626, 18627, 18628, 18629, 18630, 18631, 18632, 18633, 18634, 18635, 18636, 18637, 18638, 18639, 18640, 18641, 18642, 18643, 18644, 18645, 18646, 18647, 18648, 18649, 18650, 18651, 18652, 18653, 18654, 18655, 18656, 18657, 18658, 18659, 18660, 18661, 18662, 18663, 18664, 18665, 18666, 18667, 18668, 18669, 18670, 18671, 18672, 18673, 18674, 18675, 18676, 18677, 18678, 18679, 18680, 18681, 18682, 18683, 18684, 18685, 18686, 18687, 18688, 18689, 18690, 18691, 18692, 18693, 18694, 18695, 18696, 18697, 18698, 18699, 18700, 18701, 18702, 18703, 18704, 18705, 18706, 18707, 18708, 18709, 18710, 18711, 18712, 18713, 18714, 18715, 18716, 18717, 18718, 18719, 18720, 18721, 18722, 18723, 18724, 18725, 18726, 18727, 18728, 18729, 18730, 18731, 18732, 18733, 18734, 18735, 18736, 18737, 18738, 18739, 18740, 18741, 18742, 18743, 18744, 18745, 18746, 18747, 18748, 18749, 18750, 18751, 18752, 18753, 18754, 18755, 18756, 18757, 18758, 18759, 18760, 18761, 18762, 18763, 18764, 18765, 18766, 18767, 18768, 18769, 18770, 18771, 18772, 18773, 18774, 18775, 18776, 18777, 18778, 18779, 18780, 18781, 18782, 18783, 18784, 18785, 18786, 18787, 18788, 18789, 18790, 18791, 18792, 18793, 18794, 18795, 18796, 18797, 18798, 18799, 18800, 18801, 18802, 18803, 18804, 18805, 18806, 18807, 18808, 18809, 18810, 18811, 18812, 18813, 18814, 18815, 18816, 18817, 18818, 18819, 18820, 18821, 18822, 18823, 18824, 18825, 18826, 18827, 18828, 18829, 18830, 18831, 18832, 18833, 18834, 18835, 18836, 18837, 18838, 18839, 18840, 18841, 18842, 18843, 18844, 18845, 18846, 18847, 18848, 18849, 18850, 18851, 18852, 18853, 18854, 18855, 18856, 18857, 18858, 18859, 18860, 18861, 18862, 18863, 18864, 18865, 18866, 18867, 18868, 18869, 18870, 18871, 18872, 18873, 18874, 18875, 18876, 18877, 18878, 18879, 18880, 18881, 18882, 18883, 18884, 18885, 18886, 18887, 18888, 18889, 18890, 18891, 18892, 18893, 18894, 18895, 18896, 18897, 18898, 18899, 18900, 18901, 18902, 18903, 18904, 18905, 18906, 18907, 18908, 18909, 18910, 18911, 18912, 18913, 18914, 18915, 18916, 18917, 18918, 18919, 18920, 18921, 18922, 18923, 18924, 18925, 18926, 18927, 18928, 18929, 18930, 18931, 18932, 18933, 18934, 18935, 18936, 18937, 18938, 18939, 18940, 18941, 18942, 18943, 18944, 18945, 18946, 18947, 18948, 18949, 18950, 18951, 18952, 18953, 18954, 18955, 18956, 18957, 18958, 18959, 18960, 18961, 18962, 18963, 18964, 18965, 18966, 18967, 18968, 18969, 18970, 18971, 18972, 18973, 18974, 18975, 18976, 18977, 18978, 18979, 18980, 18981, 18982, 18983, 18984, 18985, 18986, 18987, 18988, 18989, 18990, 18991, 18992, 18993, 18994, 18995, 18996, 18997, 18998, 18999, 19000, 19001, 19002, 19003, 19004, 19005, 19006, 19007, 19008, 19009, 19010, 19011, 19012, 19013, 19014, 19015, 19016, 19017, 19018, 19019, 19020, 19021, 19022, 19023, 19024, 19025, 19026, 19027, 19028, 19029, 19030, 19031, 19032, 19033, 19034, 19035, 19036, 19037, 19038, 19039, 19040, 19041, 19042, 19043, 19044, 19045, 19046, 19047, 19048, 19049, 19050, 19051, 19052, 19053, 19054, 19055, 19056, 19057, 19058, 19059, 19060, 19061, 19062, 19063, 19064, 19065, 19066, 19067, 19068, 19069, 19070, 19071, 19072, 19073, 19074, 19075, 19076, 19077, 19078, 19079, 19080, 19081, 19082, 19083, 19084, 19085, 19086, 19087, 19088, 19089, 19090, 19091, 19092, 19093, 19094, 19095, 19096, 19097, 19098, 19099, 19100, 19101, 19102, 19103, 19104, 19105, 19106, 19107, 19108, 19109, 19110, 19111, 19112, 19113, 19114, 19115, 19116, 19117, 19118, 19119, 19120, 19121, 19122, 19123, 19124, 19125, 19126, 19127, 19128, 19129, 19130, 19131, 19132, 19133, 19134, 19135, 19136, 19137, 19138, 19139, 19140, 19141, 19142, 19143, 19144, 19145, 19146, 19147, 19148, 19149, 19150, 19151, 19152, 19153, 19154, 19155, 19156, 19157, 19158, 19159, 19160, 19161, 19162, 19163, 19164, 19165, 19166, 19167, 19168, 19169, 19170, 19171, 19172, 19173, 19174, 19175, 19176, 19177, 19178, 19179, 19180, 19181, 19182, 19183, 19184, 19185, 19186, 19187, 19188, 19189, 19190, 19191, 19192, 19193, 19194, 19195, 19196, 19197, 19198, 19199, 19200, 19201, 19202, 19203, 19204, 19205, 19206, 19207, 19208, 19209, 19210, 19211, 19212, 19213, 19214, 19215, 19216, 19217, 19218, 19219, 19220, 19221, 19222, 19223, 19224, 19225, 19226, 19227, 19228, 19229, 19230, 19231, 19232, 19233, 19234, 19235, 19236, 19237, 19238, 19239, 19240, 19241, 19242, 19243, 19244, 19245, 19246, 19247, 19248, 19249, 19250, 19251, 19252, 19253, 19254, 19255, 19256, 19257, 19258, 19259, 19260, 19261, 19262, 19263, 19264, 19265, 19266, 19267, 19268, 19269, 19270, 19271, 19272, 19273, 19274, 19275, 19276, 19277, 19278, 19279, 19280, 19281, 19282, 19283, 19284, 19285, 19286, 19287, 19288, 19289, 19290, 19291, 19292, 19293, 19294, 19295, 19296, 19297, 19298, 19299, 19300, 19301, 19302, 19303, 19304, 19305, 19306, 19307, 19308, 19309, 19310, 19311, 19312, 19313, 19314, 19315, 19316, 19317, 19318, 19319, 19320, 19321, 19322, 19323, 19324, 19325, 19326, 19327, 19328, 19329, 19330, 19331, 19332, 19333, 19334, 19335, 19336, 19337, 19338, 19339, 19340, 19341, 19342, 19343, 19344, 19345, 19346, 19347, 19348, 19349, 19350, 19351, 19352, 19353, 19354, 19355, 19356, 19357, 19358, 19359, 19360, 19361, 19362, 19363, 19364, 19365, 19366, 19367, 19368, 19369, 19370, 19371, 19372, 19373, 19374, 19375, 19376, 19377, 19378, 19379, 19380, 19381, 19382, 19383, 19384, 19385, 19386, 19387, 19388, 19389, 19390, 19391, 19392, 19393, 19394, 19395, 19396, 19397, 19398, 19399, 19400, 19401, 19402, 19403, 19404, 19405, 19406, 19407, 19408, 19409, 19410, 19411, 19412, 19413, 19414, 19415, 19416, 19417, 19418, 19419, 19420, 19421, 19422, 19423, 19424, 19425, 19426, 19427, 19428, 19429, 19430, 19431, 19432, 19433, 19434, 19435, 19436, 19437, 19438, 19439, 19440, 19441, 19442, 19443, 19444, 19445, 19446, 19447, 19448, 19449, 19450, 19451, 19452, 19453, 19454, 19455, 19456, 19457, 19458, 19459, 19460, 19461, 19462, 19463, 19464, 19465, 19466, 19467, 19468, 19469, 19470, 19471, 19472, 19473, 19474, 19475, 19476, 19477, 19478, 19479, 19480, 19481, 19482, 19483, 19484, 19485, 19486, 19487, 19488, 19489, 19490, 19491, 19492, 19493]
#print(list_idx_1stmatch)                     

In [14]:
def handle_products(x):
    x = x.strip().lower()    
    return re.sub(r'[\W_]', ' ', x).strip()  # replace '_', '-', and all non-alphanumeric with ' '
df_listings['manufacturer'] = df_listings.manufacturer.apply(handle_products)
df_listings_sorted = df_listings.sort_values('manufacturer', axis=0)  # sorted by manufacturer
df_listings_sorted.iloc[list_idx_1stmatch,:]

,title,manufacturer,price,currency
8396,Nikon COOLPIX S5 6MP Digital Camera,,399.95,USD
11918,PANASONIC DMC-LX3 BLACK,,329.00,GBP
7634,Minolta Dimage S404 4MP Digital Camera with 4x Optical Zoom,,65.00,USD
8333,Nikon Coolpix 5200 5MP Digital Camera with 3x Optical Zoom,,288.53,USD
19079,CADRE PHOTO / REVEIL / RADIO,,71.40,EUR
8332,Nikon Coolpix 5200 5MP Digital Camera with 3x Optical Zoom,,160.00,USD
11864,"SAMSUNG L201 - black Including Charger, Lithium battery",,70.00,GBP
19434,CADRE PHOTO NUMERIQUE 7 P+VID,,61.30,EUR
7595,GBA SP Digital Camera,,49.95,USD
7887,Fujifilm 12.2 Magapixels Digital Camera With 2.7 LCD Screen and 3x Optical Zoom,,109.87,USD
